In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as T
import keras
from keras import backend as K
from keras import initializers
from keras.initializers import RandomNormal
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, merge, Reshape,  Flatten, Dropout
from keras.optimizers import Adagrad, Adam, SGD, RMSprop, Adamax
from keras.regularizers import l2
from keras.layers import Multiply, Concatenate
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from time import time
import multiprocessing as mp
import sys
import math
import argparse
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [0]:
!pip install -U -q PyDrive ## you will have install for every colab session

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import os.path
import urllib.request
import tarfile
import zipfile
import gzip
from shutil import copy

def fetch_remote_datafile(filename, remote_url):
  if os.path.isfile("./" + filename):
    print("already have " + filename + " in workspace")
    return
  print("fetching " + filename + " from " + remote_url + "...")
  urllib.request.urlretrieve(remote_url, "./" + filename)

def cache_datafile_in_drive(filename):
  if os.path.isfile("./" + filename) == False:
    print("cannot cache " + filename + ", it is not in workspace")
    return
  
  data_drive_path = "./"
  if os.path.isfile(data_drive_path + filename):
    print("" + filename + " has already been stored in Google Drive")
  else:
    print("copying " + filename + " to " + data_drive_path)
    copy("./" + filename, data_drive_path)
  

def load_datafile_from_drive(filename, remote_url=None):
  data_drive_path = "./"
  if os.path.isfile("./" + filename):
    print("already have " + filename + " in workspace")
  elif os.path.isfile(data_drive_path + filename):
    print("have " + filename + " in Google Drive, copying to workspace...")
    copy(data_drive_path + filename, ".")
  elif remote_url != None:
    fetch_remote_datafile(filename, remote_url)
  else:
    print("error: you need to manually download " + filename + " and put in drive")
    
def extract_datafile(filename, expected_extract_artifact=None):
  if expected_extract_artifact != None and (os.path.isfile(expected_extract_artifact) or os.path.isdir(expected_extract_artifact)):
    print("files in " + filename + " have already been extracted")
  elif os.path.isfile("./" + filename) == False:
    print("error: cannot extract " + filename + ", it is not in the workspace")
  else:
    extension = filename.split('.')[-1]
    if extension == "zip":
      print("extracting " + filename + "...")
      data_file = open(filename, "rb")
      z = zipfile.ZipFile(data_file)
      for name in z.namelist():
          print("    extracting file", name)
          z.extract(name, "./")
      data_file.close()
    elif extension == "gz":
      print("extracting " + filename + "...")
      if filename.split('.')[-2] == "tar":
        tar = tarfile.open(filename)
        tar.extractall()
        tar.close()
      else:
        data_zip_file = gzip.GzipFile(filename, 'rb')
        data = data_zip_file.read()
        data_zip_file.close()
        extracted_file = open('.'.join(filename.split('.')[0:-1]), 'wb')
        extracted_file.write(data)
        extracted_file.close()
    elif extension == "tar":
      print("extracting " + filename + "...")
      tar = tarfile.open(filename)
      tar.extractall()
      tar.close()
    elif extension == "csv":
      print("do not need to extract csv")
    else:
      print("cannot extract " + filename)
      
def load_cache_extract_datafile(filename, expected_extract_artifact=None, remote_url=None):
  load_datafile_from_drive(filename, remote_url)
  extract_datafile(filename, expected_extract_artifact)
  cache_datafile_in_drive(filename)

In [3]:
load_cache_extract_datafile("ml-100k.zip", "ml-100k", "http://files.grouplens.org/datasets/movielens/ml-100k.zip")

fetching ml-100k.zip from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
extracting ml-100k.zip...
    extracting file ml-100k/
    extracting file ml-100k/allbut.pl
    extracting file ml-100k/mku.sh
    extracting file ml-100k/README
    extracting file ml-100k/u.data
    extracting file ml-100k/u.genre
    extracting file ml-100k/u.info
    extracting file ml-100k/u.item
    extracting file ml-100k/u.occupation
    extracting file ml-100k/u.user
    extracting file ml-100k/u1.base
    extracting file ml-100k/u1.test
    extracting file ml-100k/u2.base
    extracting file ml-100k/u2.test
    extracting file ml-100k/u3.base
    extracting file ml-100k/u3.test
    extracting file ml-100k/u4.base
    extracting file ml-100k/u4.test
    extracting file ml-100k/u5.base
    extracting file ml-100k/u5.test
    extracting file ml-100k/ua.base
    extracting file ml-100k/ua.test
    extracting file ml-100k/ub.base
    extracting file ml-100k/ub.test
ml-100k.zip has already been 

In [0]:
#MOVIELENS_DIR = '/content/ml-100k'
RATING_DATA_FILE = './ml-100k/u1.base'
RATING_TEST_FILE = './ml-100k/u1.test'
RATING_DATA_GEN_FILE = 'u1_gen.data'
RATING_DATA_GEN_TEST = 'u1_gen_test.data'
RATINGS_CSV_FILE = 'u1_emb.data'
RATINGS_CSV_FILE_TEST = 'u1_emb_test.data'
RATINGS_CSV_FILE_NORM = 'u1_normt.data'
RATINGS_CSV_FILE_NORM_TEST = 'u1_norm_test.data'
RATINGS_GEN_CSV_FILE = 'u1_genr.data'
RATINGS_GEN_CSV_FILE_TEST = 'u1_genr_test.data'
MODEL_WEIGHTS_FILE = 'u1_emb_weights.h5'
MODEL_WEIGHTS_FILE_CORE = 'u1_emb_weights'

In [0]:
rating_file_import = drive.CreateFile({'id':'1MHNyCiayXCKp6GXQMR3N9ZSNLprM2-_s'})
rating_file_import.GetContentFile(RATING_DATA_FILE)


In [7]:
ratings = pd.read_csv(RATING_DATA_FILE, 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
ratings['user_emb_id'] = ratings['userid'] - 1
ratings['movie_emb_id'] = ratings['movieid'] - 1
print(str(len(ratings))+' ratings loaded')

#save and download

ratings.to_csv(RATINGS_CSV_FILE, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['userid', 'movieid', 'rating', 'timestamp', 'user_emb_id', 'movie_emb_id'])
print('Saved to '+RATINGS_CSV_FILE)
files.download(RATINGS_CSV_FILE)

#same for test file
ratings_test = pd.read_csv(RATING_TEST_FILE, 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid_test = ratings_test['userid'].drop_duplicates().max()
max_movieid_test = ratings_test['movieid'].drop_duplicates().max()
ratings_test['user_emb_id'] = ratings_test['userid'] - 1
ratings_test['movie_emb_id'] = ratings_test['movieid'] - 1
print(str(len(ratings))+'test ratings loaded')

ratings.to_csv(RATINGS_CSV_FILE_TEST, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['userid', 'movieid', 'rating', 'timestamp', 'user_emb_id', 'movie_emb_id'])
print('Saved to '+RATINGS_CSV_FILE_TEST)
files.download(RATINGS_CSV_FILE_TEST)


100000 ratings loaded
Saved to u1_emb.data
100000test ratings loaded
Saved to u1_emb_test.data


Saved to u1_emb.data


In [8]:
ratings = pd.read_csv(RATING_DATA_FILE, 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
ratings['user_emb_id'] = ratings['userid'] - 1
ratings['movie_emb_id'] = ratings['movieid'] - 1
ratings['rating_norm'] = ratings['rating']  * 0.2
print(str(len(ratings))+' ratings loaded')
ratings.to_csv(RATINGS_CSV_FILE_NORM, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['userid', 'movieid', 'rating', 'timestamp', 'user_emb_id', 'movie_emb_id','rating_norm'])
print('Saved to '+RATINGS_CSV_FILE_NORM)
files.download(RATINGS_CSV_FILE_NORM)

#and same for TEST
ratings_test = pd.read_csv(RATING_TEST_FILE, 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid_test = ratings_test['userid'].drop_duplicates().max()
max_movieid_test = ratings_test['movieid'].drop_duplicates().max()
ratings_test['user_emb_id'] = ratings_test['userid'] - 1
ratings_test['movie_emb_id'] = ratings_test['movieid'] - 1
ratings_test['rating_norm'] = ratings_test['rating']  * 0.2
print(str(len(ratings))+' ratings loaded')
ratings.to_csv(RATINGS_CSV_FILE_NORM_TEST, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['userid', 'movieid', 'rating', 'timestamp', 'user_emb_id', 'movie_emb_id','rating_norm'])
print('Saved to '+RATINGS_CSV_FILE_NORM_TEST)
files.download(RATINGS_CSV_FILE_NORM_TEST)




100000 ratings loaded
Saved to u1_normt.data
100000 ratings loaded
Saved to u1_norm_test.data


In [9]:
#ratings are normalized between 0 to 1
ratings_norm = pd.read_csv(RATINGS_CSV_FILE_NORM, 
                      sep='\t', 
                      encoding='latin-1', 
                      usecols=['userid', 'movieid', 'user_emb_id', 'movie_emb_id', 'rating','rating_norm'])
max_userid = ratings_norm['userid'].drop_duplicates().max()
max_movieid = ratings_norm['movieid'].drop_duplicates().max()
print(str(len(ratings)), 'ratings loaded.')
print('users '+str(max_userid)+' items '+str(max_movieid))

#and same for training

ratings_norm_test = pd.read_csv(RATINGS_CSV_FILE_NORM_TEST, 
                      sep='\t', 
                      encoding='latin-1', 
                      usecols=['userid', 'movieid', 'user_emb_id', 'movie_emb_id', 'rating','rating_norm'])
max_userid_test = ratings_norm['userid'].drop_duplicates().max()
max_movieid_test = ratings_norm['movieid'].drop_duplicates().max()
print(str(len(ratings)), 'ratings loaded.')
print('users '+str(max_userid_test)+' items '+str(max_movieid_test))



100000 ratings loaded.
users 943 items 1682
100000 ratings loaded.
users 943 items 1682


In [0]:
Users = ratings['user_emb_id'].values
Movies = ratings['movie_emb_id'].values
Ratings = ratings['rating'].values
Ratings_norm = ratings_norm['rating_norm']

#same for test

Users_test = ratings_test['user_emb_id'].values
Movies_test = ratings_test['movie_emb_id'].values
Ratings_test = ratings_test['rating'].values
Ratings_norm_test = ratings_norm_test['rating_norm']

In [0]:
def get_ncf_modelv1(num_users, num_items, latent_dim,hidden_dim,do):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings
    conc = Concatenate()([user_latent, item_latent])
    drop = Dropout(0.5)(conc)
    hid1 = Dense(hidden_dim, activation='relu')(conc)
    drop2  = Dropout(do)(hid1)
    prediction = Dense(1, activation='relu', kernel_initializer='lecun_uniform', name = 'prediction')(drop2)
    
    
    model = Model(input=[user_input, item_input], output=prediction)
    print("ncf model")
    model.summary()

    return model

In [0]:
#V2

def get_ncf_modelv2(num_users, num_items, latent_dim,hidden_dim,do):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings
    conc = Concatenate()([user_latent, item_latent])
    drop = Dropout(0.5)(conc)
    hid1 = Dense(hidden_dim, activation='relu')(conc)
    #drop2  = Dropout(do)(hid1)
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(hid1)
    
    
    model = Model(input=[user_input, item_input], output=prediction)
    print("ncf model")
    model.summary()

    return model

In [0]:
#V3

def get_ncf_modelv3(num_users, num_items, latent_dim,hidden_dim,do):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings
    conc = Concatenate()([user_latent, item_latent])
    drop = Dropout(0.5)(conc)
    hid1 = Dense(hidden_dim, activation='relu')(conc)
    drop2  = Dropout(do)(hid1)
    prediction = Dense(1, activation='tanh', kernel_initializer='lecun_uniform', name = 'prediction')(drop2)
    
    
    model = Model(input=[user_input, item_input], output=prediction)
    print("ncf model")
    model.summary()

    return model

In [0]:
K_LATENT = 20
hidden_dim = 20
do = 0.5


In [16]:
#FIRST MODEL

NCF_model = get_ncf_modelv1(max_userid,max_movieid,K_LATENT,hidden_dim,do)
NCF_model.compile(loss='mse',optimizer=Adamax(),metrics=['mae'])
callbacks_ncf = [EarlyStopping('val_loss', patience=100), 
ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history_history_ncf = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100, validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)
# serialize model to JSON
# model_json = NCF_model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# NCF_model.save_weights("model_weights_forJson.h5")
# print("Saved model to disk")
 

 
 #load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model_weights_forJson.h5")
# print("Loaded model from disk")

NCF_model.evaluate([Users_test,Movies_test],Ratings_test,verbose=0)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
ncf model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 20)        18860       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedd

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("pr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 90000 samples, validate on 10000 samples
Epoch 1/100
90000/90000 [==============================] - 4s 49us/step - loss: 2.4257 - mean_absolute_error: 1.2265 - val_loss: 0.9820 - val_mean_absolute_error: 0.8103
Epoch 2/100
90000/90000 [==============================] - 4s 46us/step - loss: 1.1396 - mean_absolute_error: 0.8579 - val_loss: 0.9082 - val_mean_absolute_error: 0.7627
Epoch 3/100
90000/90000 [==============================] - 4s 45us/step - loss: 0.9579 - mean_absolute_error: 0.7833 - val_loss: 0.8941 - val_mean_absolute_error: 0.7540
Epoch 4/100
90000/90000 [==============================] - 4s 45us/step - loss: 0.9134 - mean_absolute_error: 0.7611 - val_loss: 0.8793 - val_mean_absolute_error: 0.7454
Epoch 5/100
90000/90000 [==============================] - 4s 46us/step - loss: 0.8958 - mean_absolute_error: 0.7519 - val_loss: 0.8798 - val_mean_absolute_error: 0.7440
Epoch 6/100
90000/90000 [==============================] - 4s 45us/step - loss: 0.8855 - mean_absolu

[0.7379483057022095, 0.6782954800128936]

In [17]:
#SECOND MODEL
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)


NCF_model = get_ncf_modelv1(max_userid,max_movieid,K_LATENT,hidden_dim,do)
NCF_model.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['mae'])
callbacks_ncf = [EarlyStopping('val_loss', patience=100), 
ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history_history_ncf = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100, validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)
# serialize model to JSON
# model_json = NCF_model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# NCF_model.save_weights("model_weights_forJson.h5")
# print("Saved model to disk")
 

 
 #load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model_weights_forJson.h5")
# print("Loaded model from disk")

NCF_model.evaluate([Users_test,Movies_test],Ratings_test,verbose=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("pr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


ncf model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 20)        18860       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 20)        33640       item_input[0][0]                 
__________________________________________________________________________________________________


[0.757533425450325, 0.6870894619941711]

In [18]:
#THIRD MODEL

optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)


NCF_model = get_ncf_modelv1(max_userid,max_movieid,K_LATENT,hidden_dim,do)
NCF_model.compile(loss='mean_absolute_error',optimizer=optimizer,metrics=['mae'])
callbacks_ncf = [EarlyStopping('val_loss', patience=100), 
ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history_history_ncf = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100, validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)
# serialize model to JSON
# model_json = NCF_model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# NCF_model.save_weights("model_weights_forJson.h5")
# print("Saved model to disk")
 

 
 #load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model_weights_forJson.h5")
# print("Loaded model from disk")

NCF_model.evaluate([Users_test,Movies_test],Ratings_test,verbose=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("pr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


ncf model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 20)        18860       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 20)        33640       item_input[0][0]                 
__________________________________________________________________________________________________


[2.9687247966766357, 2.9687247966766357]